<a href="https://colab.research.google.com/github/sang-ichal/SKPI-ITBU-2025/blob/main/SKPI%20ML%20Sesi%2003/SKPI_03_Sentimen_Analisis_LivinByMandiri_%5Bmodel_and_testing%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANALISIS SENTIMEN PENGGUNA TERHADAP APLIKASI LIVIN BY MANDIRI DI PLAY STORE - MODEL DAN TESTING**
**by FAIZAL RIZA [ INSTITUT TEKNOLOGI BUDI UTOMO ]**

# **1. KRITERIA**

**1.1. WAJIB**

| No  |  Kriteria | Dipenuhi  | Catatan  |
|---|---|---|---|
| 1  | Data merupakan hasil scraping dengan jumlah minimal 3000 sampel  | Ya  | Dataset hasil scraping mandiri (terlampir scraping-nya : **Scraping_Data.ipynb** dan **hasil_scraping_livin.csv**) |
| 2  | Melakukan tahapan ekstraksi fitur dan pelabelan data  | Ya  | Pada Tahap **3. Pelabelan dan Ekstraksi Fitur**  |
| 3  | Menggunakan algoritma pelatihan machine learning  | Ya  | Pada Tahap **5. Pemodelan**  |,
| 4  | Akurasi testing set yang didapatkan untuk setiap skema pelatihan minimal harus mencapai 85%  | Ya  | Dapat dilihat pada **5.6 Perbandingan Akurasi Model**  |

**1.2. TAMBAHAN**

| No  |  Kriteria | Dipenuhi  | Catatan  |
|---|---|---|---|
| 1  | Menggunakan algoritma deep learning selain Naive Bayes (NB), Random Forest (RF), Logistic Regression (LR), dan Decision Tree (DT)  | Ya  | Menggunakan **5.2 Support Vector Machine**, **5.3 LightGBM**, **5.4 XGBoost** dan **5.5 CatBoost** |
| 2  | Dataset memiliki minimal tiga kelas  | Ya | Positive, Negative dan Neutral  |
| 3  | Memiliki jumlah data minimal 10.000 sampel data  | Ya  | Data yang digunakan 150.000 sampel Cek **2.3 Dataset**  |
| 4  | Melakukan inference atau testing dalam file .ipynb yang menghasilkan output berupa kelas kategorikal (contoh : negatif, netral, dan positif)  | Ya  | Dapat dilihat pada **Testing.ipynb**  |

# **2. PREPARATION**

**2.1. Instal dan Import Library**

In [ ]:
!pip install sastrawi  # Menginstal pustaka Sastrawi untuk stemming bahasa Indonesia
!pip install tensorflow  # Menginstal TensorFlow untuk pembelajaran mesin dan deep learning
!pip install scikit-learn  # Menginstal Scikit-learn untuk alat analisis data dan pembelajaran mesin
!pip install pandas nltk wordcloud requests matplotlib seaborn # Menginstall paket lainnya
!pip install lightgbm xgboost catboost # Menginstall paket algoritma

#import library yang diperlukan
import pandas as pd  # Mengimpor pustaka pandas untuk manipulasi data
import re  # Mengimpor pustaka re untuk operasi regex (regular expression)
from nltk.tokenize import word_tokenize  # Mengimpor fungsi tokenisasi kata dari NLTK
from nltk.corpus import stopwords  # Mengimpor daftar stopwords dari NLTK
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Mengimpor pabrik Stemmer dari Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Mengimpor pabrik StopWordRemover dari Sastrawi
from wordcloud import WordCloud  # Mengimpor WordCloud untuk visualisasi kata
import nltk  # Mengimpor pustaka NLTK
nltk.download('punkt')  # Mengunduh tokenizer NLTK
nltk.download('stopwords')  # Mengunduh daftar stopwords NLTK
import requests  # Mengimpor pustaka requests untuk melakukan permintaan HTTP
import json  # Mengimpor pustaka json untuk bekerja dengan data JSON
import string  # Mengimpor pustaka string untuk operasi string
import matplotlib.pyplot as plt  # Mengimpor pyplot dari matplotlib untuk visualisasi data
from sklearn.feature_extraction.text import TfidfVectorizer  # Mengimpor TfidfVectorizer dari scikit-learn untuk ekstraksi fitur teks
from sklearn.model_selection import train_test_split  # Mengimpor fungsi pembagian data dari scikit-learn
from sklearn.metrics import accuracy_score, precision_score  # Mengimpor metrik akurasi dan presisi dari scikit-learn
import time  # Mengimpor pustaka time untuk pengukuran waktu
import numpy as np  # Mengimpor pustaka numpy untuk operasi numerik
import seaborn as sns # Import seaborn library as sns
from sklearn.svm import SVC # Support Vector Machine (SVM)
import lightgbm as lgb # LightGBM
from xgboost import XGBClassifier # XGBoost
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier # CatBoost

**2.2. Dataset**

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/sang-ichal/datasetlivin/main/hasil_scraping_livin.csv', encoding='utf-8', quotechar='"', escapechar='\\')
# Membaca file CSV dari URL ke dalam DataFrame pandas
# encoding='utf-8': Mengatur encoding karakter ke UTF-8
# quotechar='"': Mengatur karakter kutipan untuk nilai-nilai teks
# escapechar='\\': Mengatur karakter pelolosan untuk menghindari kesalahan parsing

df.head() # Menampilkan 5 baris pertama dari DataFrame

**2.3. Data Cleansing / Pembersihan Data**

In [ ]:
# Menghapus baris yang mengandung nilai null dari DataFrame dan Hasilnya disimpan ke dalam DataFrame baru bernama clean_df
clean_df = df.dropna()

# Menghapus baris duplikat dari DataFrame dan Hasilnya disimpan ke dalam DataFrame baru bernama clean_df
clean_df = clean_df.drop_duplicates()

# Menampilkan informasi ringkas tentang DataFrame clean_df Termasuk jumlah baris, kolom, tipe data masing-masing kolom, dan jumlah nilai non-null
clean_df.info()

# Menampilkan 5 baris pertama dari DataFrame
clean_df.head()

**2.4. Pra-Pemrosesan Teks / Text Preprocessing**

In [ ]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus mention (@username)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus hashtag (#hashtag)
    text = re.sub(r"http\S+", '', text)  # Menghapus URL
    text = re.sub(r'[0-9]+', '', text)  # Menghapus angka
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus karakter non-alfanumerik kecuali spasi
    text = text.replace('\n', ' ')  # Mengganti newline dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus tanda baca
    text = ' '.join([word for word in text.split() if word.lower() not in ["livin", "mandiri", "mandiri mobile"]])  # Menghapus kata-kata yang diinginkan
    text = text.strip(' ')  # Menghapus spasi di awal dan akhir teks
    return text  # Mengembalikan teks yang telah dibersihkan

def casefoldingText(text):
    text = text.lower()  # Mengubah semua karakter dalam teks menjadi huruf kecil
    return text  # Mengembalikan teks yang telah di-casefold

def tokenizingText(text):
    text = word_tokenize(text)  # Memecah teks menjadi token (kata-kata)
    return text  # Mengembalikan daftar token

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))  # Mengambil daftar stopwords bahasa Indonesia
    listStopwords1 = set(stopwords.words('english'))  # Mengambil daftar stopwords bahasa Inggris
    listStopwords.update(listStopwords1)  # Menggabungkan dua daftar stopwords
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku','di','ya','loh','kah','deh'])
    filtered = []
    for txt in text:
        if txt not in listStopwords:  # Memeriksa apakah kata bukan stopword
            filtered.append(txt)  # Menambahkan kata yang bukan stopword ke daftar filtered
    text = filtered  # Menggantikan teks dengan daftar kata yang telah difilter
    return text  # Mengembalikan daftar kata yang telah difilter

def stemmingText(text):
    factory = StemmerFactory()  # Membuat pabrik stemmer
    stemmer = factory.create_stemmer()  # Membuat stemmer
    words = text.split()  # Memecah teks menjadi kata-kata
    stemmed_words = [stemmer.stem(word) for word in words]  # Melakukan stemming pada setiap kata
    stemmed_text = ' '.join(stemmed_words)  # Menggabungkan kata-kata yang telah distem menjadi satu kalimat
    return stemmed_text  # Mengembalikan teks yang telah distem

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)  # Menggabungkan daftar kata menjadi satu kalimat
    return sentence  # Mengembalikan kalimat yang dibentuk dari daftar kata

def fix_slangwords(text):
    words = text.split()  # Memecah teks menjadi kata-kata
    fixed_words = []  # Inisialisasi daftar untuk kata-kata yang sudah diperbaiki
    for word in words:
        if word.lower() in slangwords:  # Memeriksa apakah kata tersebut adalah kata slang
            fixed_words.append(slangwords[word.lower()])  # Jika ya, mengganti kata slang dengan kata standar
        else:
            fixed_words.append(word)  # Jika bukan kata slang atau kata yang ingin dihapus, tetap mempertahankan kata tersebut
    fixed_text = ' '.join(fixed_words)  # Menggabungkan kata-kata yang sudah diperbaiki menjadi satu teks
    return fixed_text  # Mengembalikan teks yang sudah diperbaiki

**Mengambil Data Kamus Kata Baru Tidak Baku (Slang)**

In [ ]:
url = 'https://raw.githubusercontent.com/sang-ichal/datasetlivin/main/slangwords.json'  # URL tempat kamus slangwords disimpan

response = requests.get(url)  # Mengirim permintaan HTTP GET ke URL untuk mendapatkan konten

if response.status_code == 200:  # Jika status code respons adalah 200 (OK)
    try:
        slangwords = json.loads(response.text)  # Mengurai JSON yang diperoleh dari respons dan menyimpannya dalam variabel slangwords
    except json.JSONDecodeError as e: # Menangani kesalahan jika terjadi masalah saat mengurai JSON
        print("Error decoding JSON:", e)
        print("Response content:", response.text)
else:
    print("Failed to fetch data from URL. Status code:", response.status_code) # Menampilkan pesan kesalahan jika tidak berhasil mengambil data dari URL

**Prose Text Cleansing**

In [ ]:
clean_df['text_clean'] = clean_df['content'].apply(cleaningText) # Membersihkan teks pada kolom 'Content' menggunakan fungsi cleaningText dan menyimpan hasilnya dalam kolom baru 'text_clean'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText) # Melakukan case folding pada teks yang sudah dibersihkan dan menyimpan hasilnya dalam kolom 'text_casefoldingText'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords) # Memperbaiki kata-kata slang dalam teks yang sudah di-casefold dan menyimpan hasilnya dalam kolom 'text_slangwords'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText) # Melakukan tokenisasi pada teks yang sudah diperbaiki slangwords dan menyimpan hasilnya dalam kolom 'text_tokenizingText'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText) # Melakukan filtering stopwords pada teks yang sudah di-tokenisasi dan menyimpan hasilnya dalam kolom 'text_stopword'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence) # Menggabungkan kata-kata yang sudah difilter stopwords menjadi satu kalimat dan menyimpan hasilnya dalam kolom 'text_akhir'

clean_df.head() # Menampilkan 5 baris pertama dari DataFrame

**Memanggil Skor Leksikon Positif dan Negatif**

In [ ]:
def load_lexicon(url):
    lexicon = dict()  # Inisialisasi kamus untuk menyimpan kata dan skornya
    response = requests.get(url)  # Mengirim permintaan HTTP GET ke URL untuk mendapatkan konten

    if response.status_code == 200:  # Jika status code respons adalah 200 (OK)
        lines = response.text.strip().split('\n')  # Memisahkan konten menjadi baris-baris
        for line in lines:
            parts = line.split(',')  # Memisahkan baris menjadi bagian-bagian berdasarkan koma
            word = ','.join(parts[:-1])  # Menggabungkan bagian-bagian kecuali bagian terakhir (kata)
            score = parts[-1]  # Mengambil bagian terakhir sebagai skor
            lexicon[word.strip()] = int(score.strip())  # Menyimpan kata dan skornya dalam kamus, mengonversi skor ke integer
    else:
        print(f"Failed to fetch lexicon data from {url}")  # Menampilkan pesan kesalahan jika tidak berhasil mengambil data dari URL

    return lexicon  # Mengembalikan kamus yang berisi kata-kata dan skornya

positive_lexicon_url = 'https://raw.githubusercontent.com/sang-ichal/datasetlivin/main/LeksikonPositif.txt'
negative_lexicon_url = 'https://raw.githubusercontent.com/sang-ichal/datasetlivin/main/LeksikonNegatif.txt'

# Memuat leksikon positif dan negatif dari URL yang diberikan
lexicon_positive = load_lexicon(positive_lexicon_url)
lexicon_negative = load_lexicon(negative_lexicon_url)

**Penskoran Berdasarkan Akumulasi Leksikon Positif dan Negatif**

In [ ]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0  # Inisialisasi skor sentimen

    for word in text:
        if (word in lexicon_positive):  # Jika kata ada dalam leksikon positif
            score = score + lexicon_positive[word]  # Tambahkan skor positif kata tersebut ke skor total

    for word in text:
        if (word in lexicon_negative):  # Jika kata ada dalam leksikon negatif
            score = score + lexicon_negative[word]  # Tambahkan skor negatif kata tersebut ke skor total

    polarity = ''  # Inisialisasi polaritas sentimen

    if (score > 0):
        polarity = 'positive'  # Jika skor positif, tentukan polaritas positif
    elif (score < 0):
        polarity = 'negative'  # Jika skor negatif, tentukan polaritas negatif
    else:
        polarity = 'neutral'  # Jika skor nol, tentukan polaritas netral

    return score, polarity  # Mengembalikan skor sentimen dan polaritasnya

**Menentukan Polaritas Kalimat Berdasarkan Leksikon Menjadi Class Positif, Negatif atau Netral**

In [ ]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
# Menggunakan fungsi sentiment_analysis_lexicon_indonesia untuk menghitung skor sentimen dan polaritas untuk setiap teks dalam kolom 'text_stopword'
# Hasilnya disimpan dalam variabel results sebagai objek Series yang berisi tuple (skor, polaritas)

results = list(zip(*results)) # Membongkar tuple hasil menjadi dua list terpisah: satu untuk skor sentimen dan satu lagi untuk polaritas

clean_df['polarity_score'] = results[0] # Menyimpan skor sentimen dari hasil analisis ke dalam kolom baru 'polarity_score' pada DataFrame clean_df

clean_df['polarity'] = results[1] # Menyimpan polaritas sentimen dari hasil analisis ke dalam kolom baru 'polarity' pada DataFrame clean_df

print(clean_df['polarity'].value_counts()) # Mencetak jumlah kemunculan setiap kategori polaritas sentimen ('positive', 'negative', 'neutral') dari kolom 'polarity' dalam DataFrame clean_df

# clean_df.to_csv('labeled_text.csv', encoding='utf8', index=False)
#unduh dataset yang sudah diberi label

**Simpan Hasil Polaritas Leksikon**

In [ ]:
X = clean_df['text_akhir'] # Menyimpan nilai pada dataframe text_akhir ke variabel X
y = clean_df['polarity'] # Menyimpan nilai pada dataframe polarity ke variabel y

# **3.2 Ekstraksi Fitur**
Dengan ini, model akan mempertimbangkan unigram, bigram, dan trigram sebagai fitur.

Contohnya, dengan kalimat "Aplikasi Livin sangat membantu", fitur-fitur yang dihasilkan akan meliputi:

*   Unigram: "aplikasi", "livin", "sangat", "membantu"
*   Bigram: "aplikasi Livin", "Livin sangat", "sangat membantu"
*   Trigram: "aplikasi Livin sangat", "Livin sangat membantu"




In [ ]:
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8, ngram_range=(1, 1))
X_tfidf = tfidf.fit_transform(X)

Penjelasan kode :


*   **X_tfidf.toarray():** Mengubah matriks sparse X_tfidf (hasil dari TfidfVectorizer) menjadi array NumPy yang padat. Ini dilakukan agar data dapat lebih mudah diolah dan divisualisasikan.
* **tfidf.get_feature_names_out():** Mengembalikan daftar nama fitur (kata atau n-gram) yang digunakan oleh TfidfVectorizer.
*   **pd.DataFrame(...):** Membuat DataFrame Pandas dari array NumPy dan daftar nama fitur. Setiap baris dalam DataFrame mewakili dokumen (misalnya, ulasan), dan setiap kolom mewakili fitur (kata atau n-gram). Nilai dalam DataFrame adalah skor TF-IDF, yang menunjukkan seberapa penting fitur tersebut untuk dokumen tertentu.
*   **features_df:** Menyimpan DataFrame yang dihasilkan dalam variabel features_df.


In [ ]:
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
features_df

## **4. Data Visualization**

**4.1 Persentase Polaritas Tiap Sentimen**

In [ ]:
fig, ax = plt.subplots(figsize=(7, 8)) # Mengatur ukuran figure
sizes = [count for count in clean_df['polarity'].value_counts()] # Mengambil jumlah kemunculan setiap kategori polaritas sentimen dari kolom 'polarity' dalam DataFrame clean_df
#labels = list(clean_df['polarity'].value_counts().index)  # Mengambil nama kategori polaritas sentimen
labels = ['Negatif', 'Positif', 'Netral'] # Mengambil nama kategori polaritas sentimen dengan label yang sudah diubah
explode = (0, 0, 0)  # Menentukan seberapa jauh potongan (slice) akan dikeluarkan (tidak ada potongan yang dikeluarkan)
ax.pie(x=sizes, labels=labels, autopct='%1.1f%%', explode=explode, textprops={'fontsize': 14}) # Membuat pie chart dengan menggunakan data sizes dan labels
ax.set_title('Polaritas Sentimen', fontsize=20, pad=22) # Menetapkan judul pada plot
plt.show() # Menampilkan plot

**4.2 Word Cloud**

In [ ]:
positive_review = clean_df[clean_df['polarity'] == 'positive'] # Memfilter hanya data dengan polaritas positif
positive_review = positive_review[['text_akhir', 'polarity_score', 'polarity', 'text_stopword']] # Memilih kolom yang ingin ditampilkan
positive_review = positive_review.sort_values(by='polarity_score', ascending=False) # Mengurutkan berdasarkan skor polaritas secara menurun
positive_review = positive_review.reset_index(drop=True) # Mengatur ulang indeks
positive_review.index += 1 # Menambahkan nomor urut dari 1

In [ ]:
negative_review = clean_df[clean_df['polarity'] == 'negative'] # Memfilter hanya data dengan polaritas negatif
negative_review = negative_review[['text_akhir', 'polarity_score', 'polarity','text_stopword']] # Memilih kolom yang ingin ditampilkan
negative_review = negative_review.sort_values(by='polarity_score', ascending=False) # Mengurutkan berdasarkan skor polaritas secara menurun
negative_review = negative_review.reset_index(drop=True) # Mengatur ulang indeks
negative_review.index += 1 # Menambahkan nomor urut dari 1

In [ ]:
neutral_review = clean_df[clean_df['polarity'] == 'neutral'] # Memfilter hanya data dengan polaritas netral
neutral_review = neutral_review[['text_akhir', 'polarity_score', 'polarity','text_stopword']] # Memilih kolom yang ingin ditampilkan
neutral_review = neutral_review.sort_values(by='polarity_score', ascending=False) # Mengurutkan berdasarkan skor polaritas secara menurun
neutral_review = neutral_review.reset_index(drop=True) # Mengatur ulang indeks
neutral_review.index += 1 # Menambahkan nomor urut dari 1

In [ ]:
list_words = '' # Menggabungkan semua kata dari kolom 'text_stopword' menjadi satu string
for tweet in clean_df['text_stopword']:  # Mengiterasi melalui setiap tweet dalam kolom 'text_stopword' dari clean_df
    for word in tweet:  # Mengiterasi melalui setiap kata dalam setiap tweet
        list_words += ' ' + (word)  # Menambahkan kata ke dalam list_words dengan spasi di antaranya

wordcloud = WordCloud(width=600, height=400, background_color='white', min_font_size=10).generate(list_words) # Membuat WordCloud berdasarkan list_words
fig, ax = plt.subplots(figsize=(10, 8)) # Membuat figure dan axis untuk plot
ax.set_title('Word Cloud Review Semua Polarity', fontsize=20) # Menetapkan judul pada plot
ax.grid(False) # Menghilangkan grid pada plot
ax.imshow((wordcloud)) # Menampilkan WordCloud
fig.tight_layout(pad=0) # Menyesuaikan layout figure
ax.axis('off') # Menyembunyikan axis
plt.show() # Menampilkan plot

In [ ]:
list_words = '' # Menggabungkan semua kata dari kolom 'text_stopword' menjadi satu string
for review in positive_review['text_stopword']:  # Mengiterasi melalui setiap review dalam kolom 'text_stopword' dari positive_review
    for word in review:  # Mengiterasi melalui setiap kata dalam setiap review
        list_words += ' ' + (word)  # Menambahkan kata ke dalam list_words dengan spasi di antaranya

wordcloud = WordCloud(width=600, height=400, background_color='white', min_font_size=10).generate(list_words) # Membuat WordCloud berdasarkan list_words
fig, ax = plt.subplots(figsize=(10, 8)) # Membuat figure dan axis untuk plot
ax.set_title('Word Cloud Positive Review', fontsize=20) # Menetapkan judul pada plot
ax.grid(False) # Menghilangkan grid pada plot
ax.imshow((wordcloud)) # Menampilkan WordCloud
fig.tight_layout(pad=0) # Menyesuaikan layout figure
ax.axis('off') # Menyembunyikan axis
plt.show() # Menampilkan plot

In [ ]:
list_words = '' # Menggabungkan semua kata dari kolom 'text_stopword' menjadi satu string
for review in negative_review['text_stopword']:  # Mengiterasi melalui setiap review dalam kolom 'text_stopword' dari negative_review
    for word in review:  # Mengiterasi melalui setiap kata dalam setiap review
        list_words += ' ' + (word)  # Menambahkan kata ke dalam list_words dengan spasi di antaranya

wordcloud = WordCloud(width=600, height=400, background_color='white', min_font_size=10).generate(list_words) # Membuat WordCloud berdasarkan list_words
fig, ax = plt.subplots(figsize=(10, 8)) # Membuat figure dan axis untuk plot
ax.set_title('Word Cloud Negative Review', fontsize=20) # Menetapkan judul pada plot
ax.grid(False) # Menghilangkan grid pada plot
ax.imshow((wordcloud)) # Menampilkan WordCloud
fig.tight_layout(pad=0) # Menyesuaikan layout figure
ax.axis('off') # Menyembunyikan axis
plt.show() # Menampilkan plot

# **4.3 Distribusi Kelas**

In [ ]:
# Mengatur ukuran gambar plot
plt.figure(figsize=(8, 9))

# Membuat plot count menggunakan seaborn untuk variabel 'polarity' dari dataframe 'clean_df'
class_dist_plot = sns.countplot(x='polarity', data=clean_df)

# Menetapkan judul plot
plt.title('Distribusi Kelas')

# Menetapkan label sumbu x
plt.xlabel('Polarity')

# Menetapkan label sumbu y
plt.ylabel('Count')

# Menambahkan label angka untuk setiap bar pada plot
for p in class_dist_plot.patches:
    class_dist_plot.annotate(format(p.get_height(), '.0f'),
                             (p.get_x() + p.get_width() / 2., p.get_height()),
                             ha='center', va='center',
                             xytext=(0, 10),
                             textcoords='offset points')

# Menampilkan plot
plt.show()

# **5. Pemodelan**

## **5.1 Data Splitting**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

## **5.2 Support Vector Machine**

In [ ]:
# Membuat objek model SVM
svm = SVC(kernel='linear')

# Melatih model SVM pada data pelatihan
start_time = time.time()
svm.fit(X_train.toarray(), y_train)
execution_time = time.time() - start_time

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_svm = svm.predict(X_train.toarray())
y_pred_test_svm = svm.predict(X_test.toarray())

# Evaluasi akurasi model SVM pada data pelatihan
accuracy_train_svm = accuracy_score(y_pred_train_svm, y_train)

# Evaluasi akurasi model SVM pada data uji
accuracy_test_svm = accuracy_score(y_pred_test_svm, y_test)

# Menampilkan akurasi dan waktu eksekusi
print('SVM - accuracy_train:', accuracy_train_svm)
print('SVM - accuracy_test:', accuracy_test_svm)
print('SVM - execution_time:', execution_time, 'seconds')

## **5.3 LightGBM**

In [ ]:
# Membuat objek model LightGBM
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)

# Melatih model LightGBM pada data pelatihan
start_time = time.time()
lgb_model.fit(X_train.toarray(), y_train)
execution_time = time.time() - start_time

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lgb = lgb_model.predict(X_train.toarray())
y_pred_test_lgb = lgb_model.predict(X_test.toarray())

# Evaluasi akurasi model LightGBM pada data pelatihan
accuracy_train_lgb = accuracy_score(y_pred_train_lgb, y_train)

# Evaluasi akurasi model LightGBM pada data uji
accuracy_test_lgb = accuracy_score(y_pred_test_lgb, y_test)

# Menampilkan akurasi dan waktu eksekusi
print('LightGBM - accuracy_train:', accuracy_train_lgb)
print('LightGBM - accuracy_test:', accuracy_test_lgb)
print('LightGBM - execution_time:', execution_time, 'seconds')

## **5.4 XGBoost**

In [ ]:
# Mengonversi sparse matrix ke array
X_train_array = X_train.toarray()
X_test_array = X_test.toarray()

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Membuat objek model XGBoost
xgb = XGBClassifier(n_estimators=100, random_state=42)

# Melatih model XGBoost pada data pelatihan
start_time = time.time()
xgb.fit(X_train_array, y_train_encoded)
execution_time = time.time() - start_time

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_xgb = xgb.predict(X_train_array)
y_pred_test_xgb = xgb.predict(X_test_array)

# Evaluasi akurasi model XGBoost pada data pelatihan
accuracy_train_xgb = accuracy_score(y_pred_train_xgb, y_train_encoded)

# Evaluasi akurasi model XGBoost pada data uji
accuracy_test_xgb = accuracy_score(y_pred_test_xgb, y_test_encoded)

# Menampilkan akurasi dan waktu eksekusi
print('XGBoost - accuracy_train:', accuracy_train_xgb)
print('XGBoost - accuracy_test:', accuracy_test_xgb)
print('XGBoost - execution_time:', execution_time, 'seconds')

## **5.5 CatBoost**

In [ ]:
from catboost import CatBoostClassifier
import time

# Membuat objek model CatBoost
cat_model = CatBoostClassifier(n_estimators=100, random_state=42, verbose=0)

# Melatih model CatBoost pada data pelatihan
start_time = time.time()
cat_model.fit(X_train.toarray(), y_train)
execution_time = time.time() - start_time

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_cat = cat_model.predict(X_train.toarray())
y_pred_test_cat = cat_model.predict(X_test.toarray())

# Evaluasi akurasi model CatBoost pada data pelatihan
accuracy_train_cat = accuracy_score(y_pred_train_cat, y_train)

# Evaluasi akurasi model CatBoost pada data uji
accuracy_test_cat = accuracy_score(y_pred_test_cat, y_test)

# Menampilkan akurasi dan waktu eksekusi
print('CatBoost - accuracy_train:', accuracy_train_cat)
print('CatBoost - accuracy_test:', accuracy_test_cat)
print('CatBoost - execution_time:', execution_time, 'seconds')

## **5.6 Perbandingan Akurasi Model**

In [ ]:
# Membuat DataFrame untuk hasil akurasi
results_df = pd.DataFrame({
    'Model': ['Support Vector Machine','Light Gradient-Boosting Machine','Extreme Gradient Boosting','Categorical Boosting'],
    'Accuracy Train': [accuracy_train_svm, accuracy_train_lgb, accuracy_train_xgb, accuracy_train_cat],
    'Accuracy Test': [accuracy_test_svm, accuracy_test_lgb, accuracy_test_xgb, accuracy_test_cat]
})

# Menampilkan hanya kolom "Accuracy Test"
accuracy_test_only = results_df[['Model', 'Accuracy Test']]

# Mengurutkan DataFrame berdasarkan kolom "Accuracy Test" dari tertinggi ke terendah
accuracy_test_sorted = accuracy_test_only.sort_values(by='Accuracy Test', ascending=False)

# Menampilkan DataFrame yang telah diurutkan
print(accuracy_test_sorted)

# **6. KESIMPULAN**

Berdasarkan perbandingan tersebut terlihat Support Vector Machine memiliki akurasi paling tinggi dari algoritma lainnya yaitu 87.13%.

Selanjutnya, pemodelan akan digunakan untuk pengetesan inference dengan pertama-tama Menyimpan Model yang Terlatih dengan akurasi paling tinggi dan Objek TF-IDF di **Pemodelan.ipynb**, dan berikutnya Memuat Model yang Terlatih dan Objek TF-IDF di **Testing.ipynb**.

In [ ]:
import pickle

# Simpan model SVM
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm, f)

In [ ]:
# Simpan TfidfVectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# **-----------------------------------------------------------------------------**

# **TESTING MODEL**

### **1. PREPARATION**

In [ ]:
!pip install Sastrawi
!pip install nltk
import pickle
from pickle import UnpicklingError
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import requests
import json
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Muat model SVM
try:
    with open('/content/svm_model.pkl', 'rb') as f:
        svm = pickle.load(f)
except (EOFError, UnpicklingError) as e:
    print(f"Error loading SVM model: {e}")
    # Additional diagnostic steps:
    print("Periksa kembali apakah 'svm_model.pkl' ada dan tidak kosong.")
    print("Jika file model berasal dari transfer data, pastikan sudah berada pada binary mode.")

# Muat TfidfVectorizer
try:
    with open('/content/tfidf_vectorizer.pkl', 'rb') as f:
        tfidf = pickle.load(f)
except (EOFError, UnpicklingError) as e:
    print(f"Error loading TF-IDF vectorizer: {e}")

# Fungsi pra-pemrosesan teks (sama seperti di Pemodelan.ipynb)
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus mention (@username)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus hashtag (#hashtag)
    text = re.sub(r"http\S+", '', text)  # Menghapus URL
    text = re.sub(r'[0-9]+', '', text)  # Menghapus angka
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus karakter non-alfanumerik kecuali spasi
    text = text.replace('\n', ' ')  # Mengganti newline dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus tanda baca
    text = ' '.join([word for word in text.split() if word.lower() not in ["livin", "mandiri", "mandiri mobile"]])  # Menghapus kata-kata yang diinginkan
    text = text.strip(' ')  # Menghapus spasi di awal dan akhir teks
    return text  # Mengembalikan teks yang telah dibersihkan

def casefoldingText(text):
    text = text.lower()  # Mengubah semua karakter dalam teks menjadi huruf kecil
    return text  # Mengembalikan teks yang telah di-casefold

def tokenizingText(text):
    text = word_tokenize(text)  # Memecah teks menjadi token (kata-kata)
    return text  # Mengembalikan daftar token

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))  # Mengambil daftar stopwords bahasa Indonesia
    listStopwords1 = set(stopwords.words('english'))  # Mengambil daftar stopwords bahasa Inggris
    listStopwords.update(listStopwords1)  # Menggabungkan dua daftar stopwords
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku','di','ya','loh','kah','deh'])
    filtered = []
    for txt in text:
        if txt not in listStopwords:  # Memeriksa apakah kata bukan stopword
            filtered.append(txt)  # Menambahkan kata yang bukan stopword ke daftar filtered
    text = filtered  # Menggantikan teks dengan daftar kata yang telah difilter
    return text  # Mengembalikan daftar kata yang telah difilter

def stemmingText(text):
    factory = StemmerFactory()  # Membuat pabrik stemmer
    stemmer = factory.create_stemmer()  # Membuat stemmer
    words = text.split()  # Memecah teks menjadi kata-kata
    stemmed_words = [stemmer.stem(word) for word in words]  # Melakukan stemming pada setiap kata
    stemmed_text = ' '.join(stemmed_words)  # Menggabungkan kata-kata yang telah distem menjadi satu kalimat
    return stemmed_text  # Mengembalikan teks yang telah distem

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)  # Menggabungkan daftar kata menjadi satu kalimat
    return sentence  # Mengembalikan kalimat yang dibentuk dari daftar kata

def fix_slangwords(text):
    words = text.split()  # Memecah teks menjadi kata-kata
    fixed_words = []  # Inisialisasi daftar untuk kata-kata yang sudah diperbaiki
    for word in words:
        if word.lower() in slangwords:  # Memeriksa apakah kata tersebut adalah kata slang
            fixed_words.append(slangwords[word.lower()])  # Jika ya, mengganti kata slang dengan kata standar
        else:
            fixed_words.append(word)  # Jika bukan kata slang atau kata yang ingin dihapus, tetap mempertahankan kata tersebut
    fixed_text = ' '.join(fixed_words)  # Menggabungkan kata-kata yang sudah diperbaiki menjadi satu teks
    return fixed_text  # Mengembalikan teks yang sudah diperbaiki

url = 'https://raw.githubusercontent.com/sang-ichal/datasetlivin/main/slangwords.json'  # URL tempat kamus slangwords disimpan

response = requests.get(url)  # Mengirim permintaan HTTP GET ke URL untuk mendapatkan konten

if response.status_code == 200:  # Jika status code respons adalah 200 (OK)
    try:
        slangwords = json.loads(response.text)  # Mengurai JSON yang diperoleh dari respons dan menyimpannya dalam variabel slangwords
    except json.JSONDecodeError as e: # Menangani kesalahan jika terjadi masalah saat mengurai JSON
        print("Error decoding JSON:", e)
        print("Response content:", response.text)
else:
    print("Failed to fetch data from URL. Status code:", response.status_code) # Menampilkan pesan kesalahan jika tidak berhasil mengambil data dari URL

def preprocess_text(text):
    text = cleaningText(text)
    text = casefoldingText(text)
    text = fix_slangwords(text)
    text = tokenizingText(text)
    text = filteringText(text)
    text = toSentence(text)
    return text

def prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm):
    # Melakukan preprocessing pada kalimat baru
    review_baru_cleaned = cleaningText(review_baru)
    review_baru_casefolded = casefoldingText(review_baru_cleaned)
    review_baru_slangfixed = fix_slangwords(review_baru_casefolded)
    review_baru_tokenized = tokenizingText(review_baru_slangfixed)
    review_baru_filtered = filteringText(review_baru_tokenized)
    review_baru_final = toSentence(review_baru_filtered)

    # Menggunakan objek tfidf yang sudah di-fit dari pelatihan sebelumnya
    X_review_baru = tfidf.transform([review_baru_final])

    # Convert sparse matrix to dense array
    X_review_baru = X_review_baru.toarray()

    # Memperoleh prediksi sentimen review baru menggunakan model terbaik
    prediksi_sentimen = svm.predict(X_review_baru)

    # Menampilkan hasil prediksi
    if prediksi_sentimen[0] == 'positive':
        hasil = "Sentimen review baru adalah POSITIF."
    elif prediksi_sentimen[0] == 'negative':
        hasil = "Sentimen review baru adalah NEGATIF."
    else:
        hasil = "Sentimen review baru adalah NETRAL."

    return hasil

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **2 Kategori Negatif**

In [ ]:
review_baru = "hang mulu loginnya nih apps gagal mulu"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah NEGATIF.'

### **3 Kategori Positif**

In [ ]:
review_baru = "Aplikasinya mantap, membantu transaksi saya tdk pernah gagal"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah POSITIF.'

### **4. Kategori Netral**

In [ ]:
review_baru = "Sudah oke"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)


'Sentimen review baru adalah NETRAL.'

**REQUIREMENT.TXT**

In [ ]:
pip freeze > /content/requirements.txt

# **6. KESIMPULAN**

Berdasarkan perbandingan tersebut terlihat [algoritma ] memiliki akurasi paling tinggi dari algoritma lainnya yaitu ...... %.

Selanjutnya, pemodelan akan digunakan untuk pengetesan inference dengan pertama-tama Menyimpan Model yang Terlatih dengan akurasi paling tinggi dan Objek TF-IDF di **Pemodelan.ipynb**, dan berikutnya Memuat Model yang Terlatih dan Objek TF-IDF di **Testing.ipynb**.

In [ ]:
import pickle

# Simpan model SVM
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm, f)

In [ ]:
# Simpan TfidfVectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [ ]:
X = clean_df['text_akhir'] # Menyimpan nilai pada dataframe text_akhir ke variabel X
y = clean_df['polarity'] # Menyimpan nilai pada dataframe polarity ke variabel y